# Decision Tree

In [1]:
import numpy as np
import pandas as pd
from sklearn import pipeline
from sklearn import tree
from sklearn import preprocessing
from sklearn import metrics
from sklearn import impute
from sklearn import model_selection
from matplotlib import pyplot as plt

# Pre-processing

## Reading csv


In [30]:
dataset = pd.read_csv("./dataset.csv")
dataset.drop(["Unnamed: 0", "track_id", "track_name", "album_name"], axis=1, inplace=True)

## Show data

In [31]:
print(dataset.shape)
print()
dataset.info()
dataset.head()

(114000, 17)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artists           113999 non-null  object 
 1   popularity        114000 non-null  int64  
 2   duration_ms       114000 non-null  int64  
 3   explicit          114000 non-null  bool   
 4   danceability      114000 non-null  float64
 5   energy            114000 non-null  float64
 6   key               114000 non-null  int64  
 7   loudness          114000 non-null  float64
 8   mode              114000 non-null  int64  
 9   speechiness       114000 non-null  float64
 10  acousticness      114000 non-null  float64
 11  instrumentalness  114000 non-null  float64
 12  liveness          114000 non-null  float64
 13  valence           114000 non-null  float64
 14  tempo             114000 non-null  float64
 15  time_signature    114000 non-null  int64  
 16  track_

,artists,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,Gen Hoshino,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,Ben Woodward,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,Ingrid Michaelson;ZAYN,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,Kina Grannis,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,Chord Overstreet,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


## Clean data

In [32]:
def train_validation_test_split(
    df, target_column, validation_size=0.1, test_size=0.1, random_state=42
):
    df_train, df_test = model_selection.train_test_split(
        df, test_size=test_size, random_state=random_state, stratify=df[target_column]
    )

    df_train, df_validation = model_selection.train_test_split(
        df_train,
        test_size=validation_size / (1 - test_size),
        random_state=random_state,
        stratify=df_train[target_column],
    )
    return df_train, df_validation, df_test


random_state = 42

df_train, df_validation, df_test = train_validation_test_split(
    df=dataset,
    target_column="track_genre",
    validation_size=0.2,
    test_size=0.2,
    random_state=random_state,
)

assert df_train.shape[0] + df_validation.shape[0] + df_test.shape[0] == dataset.shape[0]

## Adding median values to NaN

In [33]:
numeric_columns = df_train.select_dtypes(include=['number']).columns

numeric_imputer = impute.SimpleImputer(strategy="median")

numeric_imputer.fit(df_train[numeric_columns])

df_train[numeric_columns] = numeric_imputer.transform(df_train[numeric_columns])
df_validation[numeric_columns] = numeric_imputer.transform(df_validation[numeric_columns])
df_test[numeric_columns] = numeric_imputer.transform(df_test[numeric_columns])

In [34]:
df_train.dropna(inplace=True, how="any")
df_test.dropna(inplace=True, how="any")
df_validation.dropna(inplace=True, how="any")

## Normalizing data

In [35]:
normalizer = preprocessing.MinMaxScaler()

normalizer.fit(df_train[numeric_columns])

df_train[numeric_columns] = normalizer.transform(df_train[numeric_columns])
df_validation[numeric_columns] = normalizer.transform(df_validation[numeric_columns])
df_test[numeric_columns] = normalizer.transform(df_test[numeric_columns])

## Adding encoder

In [40]:
categorical_columns = df_train.iloc[:, :-1].select_dtypes(include=["object"]).columns

print(categorical_columns)
target_encoder = preprocessing.LabelEncoder()

target_encoder.fit(df_train[categorical_columns])

print(df_train[categorical_columns])
print(target_encoder.transform(df_train[categorical_columns]))
df_train[categorical_columns] = target_encoder.transform(
    df_train[categorical_columns].ravel()
)
df_validation[categorical_columns] = target_encoder.transform(
    df_validation[categorical_columns].ravel()
)
df_test[categorical_columns] = target_encoder.transform(
    df_test[categorical_columns].ravel()
)

Index(['artists'], dtype='object')
                                artists
14769                       The Beanies
97099                Henrique & Juliano
82855  Elvis Costello & The Attractions
36654      Kid Francescoli;Julia Minkin
57729                       salem ilese
...                                 ...
32477             MYRNE;JPB;Yung Fusion
11298                       Nick Mulvey
57400                  Labrinth;Zendaya
17065                   Le Castle Vania
84432                           The Who

[68399 rows x 1 columns]
[20100  8706  6494 ... 11545 11705 20695]


c:\Users\Marcos\OneDrive\Desktop\Meus Projetos\AM-msc\Projetos-AM\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Marcos\OneDrive\Desktop\Meus Projetos\AM-msc\Projetos-AM\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


AttributeError: 'DataFrame' object has no attribute 'ravel'

## Training

In [44]:
decision_tree = tree.DecisionTreeClassifier(random_state=random_state)

parameters = {
    "criterion": ["gini", "entropy", "log_loss"],
    "splitter": ["best", "random"],
    "max_depth": np.arange(15),
    "max_features": ["sqrt", "log2"],
    "max_leaf_nodes": np.arange(2, 15),
}

grid_search = model_selection.RandomizedSearchCV(
    cv=5,
    random_state=random_state,
    n_iter=15,
    n_jobs=-1,
    estimator=decision_tree,
    param_distributions=parameters,
)

grid_search.fit(df_train.iloc[:, :-1], df_train.iloc[:, -1])

ValueError: 
All the 75 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Marcos\OneDrive\Desktop\Meus Projetos\AM-msc\Projetos-AM\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Marcos\OneDrive\Desktop\Meus Projetos\AM-msc\Projetos-AM\.venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Marcos\OneDrive\Desktop\Meus Projetos\AM-msc\Projetos-AM\.venv\Lib\site-packages\sklearn\tree\_classes.py", line 1009, in fit
    super()._fit(
  File "c:\Users\Marcos\OneDriv